In [94]:
from binance import Client
from datetime import datetime, timedelta
client = Client(api_key="jM9zEeTTzT7R7QGlNrh5tUpOc8l71hOxu7CewCL4M0NvbBp3ilgmEwMOOjJfCRVB",api_secret="iZxFmlz5bvROsHQPZTP5hlJtsxYjOu8wocOlLWzIDdhMzEQu7KyyHB2DseOYQ66s",testnet=True)

import os
from google.cloud.sql.connector import Connector
from sqlalchemy import create_engine, text
import pg8000.native
import pandas as pd
from time import sleep

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tinma/OneDrive/Escritorio/application_default_credentials.json'

connector = Connector()

def getconn() -> pg8000.native.Connection:
    conn: pg8000.native.Connection = connector.connect(
        "semiotic-pact-388619:us-central1:tradingbot",
        "pg8000",
        user="postgres",
        password="GuruSat.3",
        db="criptos"
    )
    return conn

connection = create_engine('postgresql+pg8000://',creator=getconn)

def MACD(df):
    df['EMA12'] = df.Close.ewm(span=12).mean()
    df['EMA26'] = df.Close.ewm(span=26).mean()
    df['MACD'] = df.EMA12 - df.EMA26
    df['Signal'] = df.MACD.ewm(span=9).mean()
    df['Histogram'] = df.MACD - df.Signal
    # print('Indicadores agregados')

def señal_MACD_wzero(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0 and df.MACD < 0 and df.Signal < 0:
            buy.append(i)
        elif df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0 and df.MACD > 0 and df.Signal > 0:
            sell.append(i)
    return buy, sell

def señal_MACD(df):
    buy, sell = [],[]
    for i in range(7,len(df)):
        if df.Histogram.iloc[i] > 0 and df.Histogram.iloc[i-1] < 0:
            buy.append(i)
        elif df.Histogram.iloc[i] < 0 and df.Histogram.iloc[i-1] > 0:
            sell.append(i)
    return buy, sell

In [74]:
df = pd.DataFrame(client.get_historical_klines('BTCUSDT','1m','30 m ago UTC'))
df = df.iloc[:,:6]
df.columns = ['Tiempo','Open','High','Low','Close','Volume']
df.set_index('Tiempo',inplace=True)
df = df[['Close']]
df.index = pd.to_datetime(df.index,unit='ms')
df.sort_values(by='Tiempo',inplace=True)
df

,Close
Tiempo,
2023-07-05 19:30:00,30504.50000000
2023-07-05 19:31:00,30518.13000000
2023-07-05 19:32:00,30522.86000000
2023-07-05 19:33:00,30515.92000000
2023-07-05 19:34:00,30518.61000000
2023-07-05 19:35:00,30512.17000000
2023-07-05 19:36:00,30522.41000000
2023-07-05 19:37:00,30527.58000000
2023-07-05 19:38:00,30526.30000000


In [81]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1688617091784,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.00500000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.20000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '9467.49616930', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': 

In [96]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Trades/MACD/by1m/'
balances = []
PAIR = 'BTCUSDT'
asset = 'btc'
LOCALIDAD_SEÑAL_ENTRADA = 30 # SEGUNDOS
LOCALIDAD_SEÑAL_SALIDA = 30 # SEGUNDOS
UMBRAL_PERDIDA = - 30 # DÓLARES
open_order = False
balance_trade_anterior = 0
balance_trade = 0
open_order = False
tiempo_inicial = datetime.utcnow()
estrategia = 'MACD'
cantidad_trade = 0.005

while True:

    if datetime.utcnow().second == 5:

######################## STREAMING Y ESTRATEGIA.

        df = pd.DataFrame(client.get_historical_klines('BTCUSDT','1m','1 day ago UTC'))
        df = df.iloc[:,:6]
        df.columns = ['Tiempo','Open','High','Low','Close','Volume']
        df.set_index('Tiempo',inplace=True)
        df = df[['Close']]
        df['Close'] = df.Close.astype(float)
        df.index = pd.to_datetime(df.index,unit='ms')
        df.sort_values(by='Tiempo',inplace=True)
        df
        MACD(df)
        buy, sell = señal_MACD(df)

        buyprices = df.Close.iloc[buy]
        sellprices = df.Close.iloc[sell]

######################## MÉTODO DE ENTRADA POR MEDIO DE SEÑAL.

        if open_order == False:

            DIFF_SEÑAL_ENTRADA = (datetime.utcnow() - buyprices.index[-1]).total_seconds()

            if DIFF_SEÑAL_ENTRADA < LOCALIDAD_SEÑAL_ENTRADA:
                open_order = True
                buy_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_BUY,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade,
                            recvWindow=60000
                        )
                tiempo_entrada = pd.to_datetime(buy_order['transactTime'], unit='ms')
                precio_entrada = float(buy_order['fills'][0]['price'])
                print(f'Se realizó una compra al precio: {precio_entrada}, en el momento: {tiempo_entrada}')

        if open_order == True:

######################## MÉTODO DE SALIDA POR MEDIO DEL UMBRAL DE PÉRIDA.

            if df.Close[-1] - precio_entrada < UMBRAL_PERDIDA:
                open_order = False
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade,
                            recvWindow=60000
                        )
                Metodo = 'UMBRAL_PERDIDA'
                tiempo_salida = pd.to_datetime(sell_order['transactTime'], unit='ms')
                precio_salida = float(sell_order['fills'][0]['price'])
                balance_trade = (precio_salida - precio_entrada)*cantidad_trade
                balances.append(balance_trade)
                
######################## MÉTODO DE SALIDA POR MEDIO DE SEÑAL

        if open_order == True:

            DIFF_SEÑAL_SALIDA = (datetime.utcnow() - sellprices.index[-1]).total_seconds()

            if DIFF_SEÑAL_SALIDA < LOCALIDAD_SEÑAL_SALIDA and sellprices.index[-1] > buyprices.index[-1]:
                open_order = False
                sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade,
                            recvWindow=60000
                        )
                Metodo = 'SEÑAL'
                tiempo_salida = pd.to_datetime(sell_order['transactTime'], unit='ms')
                precio_salida = float(sell_order['fills'][0]['price'])
                balance_trade = (precio_salida - precio_entrada)*cantidad_trade
                balances.append(balance_trade)
                        
######################## LOGS

        if balance_trade_anterior != balance_trade:
            balance_trade_anterior = balance_trade
            if balance_trade>0:
                print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Ganancia de: {balance_trade} dólares. \nMétodo: {Metodo}')
                trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade)]],
                                    columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Ganancia'])
                if os.path.exists(path+f'trades_ganados_{asset}.csv'):
                    trade_log.to_csv(path+f'trades_ganados_{asset}.csv',
                                    mode='a',
                                    index=False,
                                    header=False
                                    )
                else:
                    trade_log.to_csv(path+f'trades_ganados_{asset}.csv',
                                    index=False,
                                    header=True,
                                    )
            elif balance_trade==0:
                print('No hubo pérdidas ni ganancias del trade')
            else:
                print(f'Se realizó una venta al precio: {precio_salida}, en el momento: {tiempo_salida}. Pérdida de: {balance_trade} dólares. \nMétodo: {Metodo}')
                trade_log = pd.DataFrame([[precio_entrada, tiempo_entrada, precio_salida, tiempo_salida, (balance_trade),Metodo]],
                                    columns=['Precio_Entrada','Tiempo_Entrada','Precio_Salida','Tiempo_Salida','Perdida','Metodo'])
                if os.path.exists(path+f'trades_perdidos_{asset}.csv'):
                    trade_log.to_csv(path+f'trades_perdidos_{asset}.csv',
                                    mode='a',
                                    index=False,
                                    header=False
                                    )
                else:
                    trade_log.to_csv(path+f'trades_perdidos_{asset}.csv',
                                    index=False,
                                    header=True,
                                    )

Se realizó una compra al precio: 29975.71, en el momento: 2023-07-06 23:25:08.144000
Se realizó una venta al precio: 29919.96, en el momento: 2023-07-06 23:57:07.752000. Pérdida de: -0.27875 dólares. 
Método: UMBRAL_PERDIDA
Se realizó una compra al precio: 29857.3, en el momento: 2023-07-07 00:10:08.166000
Se realizó una venta al precio: 29966.76, en el momento: 2023-07-07 00:30:07.944000. Ganancia de: 0.5472999999999957 dólares. 
Método: SEÑAL
Se realizó una compra al precio: 29953.47, en el momento: 2023-07-07 00:42:08.203000
Se realizó una venta al precio: 30017.2, en el momento: 2023-07-07 00:53:07.647000. Ganancia de: 0.3186499999999978 dólares. 
Método: SEÑAL
Se realizó una compra al precio: 30011.79, en el momento: 2023-07-07 01:30:07.933000
Se realizó una venta al precio: 30069.28, en el momento: 2023-07-07 01:47:07.544000. Ganancia de: 0.2874499999999898 dólares. 
Método: SEÑAL
Se realizó una compra al precio: 30120.22, en el momento: 2023-07-07 02:45:08.210000
Se realizó una 

KeyboardInterrupt: 

In [97]:
path = 'C:/Users/tinma/OneDrive/Escritorio/tradingbot/trade_logs/testnet/logs/Sesiones/MACD_MACD+logicv1/by1m/'

# sleep(5)

if open_order:
    sell_order = client.create_order(
                            symbol=PAIR,
                            side=Client.SIDE_SELL,
                            type=Client.ORDER_TYPE_MARKET,
                            quantity=cantidad_trade
                        )
    open_order = False
    tiempo_salida = pd.to_datetime(sell_order['transactTime'], unit='ms')
    precio_salida = float(sell_order['fills'][0]['price'])
    balance_trade = (precio_salida - precio_entrada)*cantidad_trade
    balances.append(balance_trade)

if tiempo_entrada > tiempo_salida:
    tiempo_final = tiempo_entrada
elif tiempo_entrada < tiempo_salida:
    tiempo_final = tiempo_salida

if sum(balances) > 0:
    session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Ganancia','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Ganadas.csv'):
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Ganadas.csv',
                        index=False,
                        header=True
                        )
elif sum(balances) < 0:
    session_log = pd.DataFrame([[len(balances), sum(balances), sum(balances)/len(balances), PAIR, estrategia, tiempo_inicial, tiempo_final,cantidad_trade]],
             columns=['Cantidad_Trades','Perdida','Ganancia_Promedio_xTrade','PAR','Estrategia','Tiempo_inicial','Tiempo_final','Cantidad_trade'])
    if os.path.exists(path+'Sesiones_Perdidas.csv'):
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        mode='a',
                        index=False,
                        header=False
                        )
    else:
        session_log.to_csv(path+'Sesiones_Perdidas.csv',
                        index=False,
                        header=True
                        )

In [98]:
session_log

,Cantidad_Trades,Ganancia,Ganancia_Promedio_xTrade,PAR,Estrategia,Tiempo_inicial,Tiempo_final,Cantidad_trade
0,12,0.2348,0.019567,BTCUSDT,MACD,2023-07-06 22:57:02.888723,2023-07-07 06:46:08.300,0.005


1.0573500000000102

In [90]:
pd.to_datetime(sell_order['transactTime'], unit='ms')

Timestamp('2023-07-06 20:10:10.209000')